In [1]:
%load_ext pycodestyle_magic
%load_ext mypy_ipython
%pycodestyle_on

In [2]:
import doctest

In [3]:
import sys
import importlib
from collections import defaultdict

_post_import_hooks = defaultdict(list)


class PostImportFinder:
    def __init__(self):
        self._skip = set()

    def find_module(self, fullname, path=None):
        if fullname in self._skip:
            return None

        self._skip.add(fullname)
        return PostImportLoader(self)


class PostImportLoader:
    def __init__(self, finder):
        self._finder = finder

    def load_module(self, fullname):
        importlib.import_module(fullname)
        module = sys.modules[fullname]
        for func in _post_import_hooks[fullname]:
            func(module)

        self._finder._skip.remove(fullname)
        return module


def when_imported(fullname):
    def decorate(func):
        if fullname in sys.modules:
            func(sys.modules[fullname])
        else:
            _post_import_hooks[fullname].append(func)

        return func
    return decorate


sys.meta_path.insert(0, PostImportFinder())


"""
Basic

    >>> 'threading' in sys.modules
    True
    >>> @when_imported('threading')
    ... def warn_threads(mod):
    ...     print("i'm not thread-safe")
    i'm not thread-safe

More practical example

    >>> from functools import wraps
    >>> import sys
    >>> def logged(func):
    ...     @wraps(func)
    ...     def wrapper(*args, **kwargs):
    ...         print('calling', func.__name__, args, kwargs)
    ...         return func(*args, **kwargs)
    ...     return wrapper
    >>> @when_imported('math')
    ... def add_logging(mod):
    ...     mod.cos = logged(mod.cos)
    ...     mod.sin = logged(mod.sin)
    >>> import math
    >>> math.sin(1)
    calling sin (1,) {}
    0.8414709848078965

"""

doctest.testmod()

TestResults(failed=0, attempted=8)